<a href="https://colab.research.google.com/github/edtorrente/big-data-challenge/blob/main/level2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backpor

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-02-25 19:57:19--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.23MB/s    in 0.2s    

2022-02-25 19:57:19 (5.23 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Video").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
# Load in employee.csv from S3 into a DataFrame
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

video_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Video_v1_00.tsv.gz"), inferSchema=True, sep='\t', header=True)
video_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   49033728|R1P1G5KZ05H6RD|6302503213|     748506413|The Night They Sa...|           Video|          5|            0|          0|   N|                Y|    Very satisfied!!|Fast shipping. Pl...| 2015-08-31|
|         US|   17857748|R106N066IUN8ZV|B000059PET|     478710180|Hamlet / Kline, N...|           Video|          5|    

In [5]:
video_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



##filter by votes

In [8]:


df2= video_df.select(["helpful_votes", "star_rating", "total_votes", "vine", "verified_purchase"])

df2.show()

+-------------+-----------+-----------+----+-----------------+
|helpful_votes|star_rating|total_votes|vine|verified_purchase|
+-------------+-----------+-----------+----+-----------------+
|            0|          5|          0|   N|                Y|
|            0|          5|          0|   N|                Y|
|            0|          4|          0|   N|                Y|
|            0|          5|          0|   N|                Y|
|            0|          3|          0|   N|                N|
|            0|          5|          0|   N|                Y|
|            0|          4|          0|   N|                Y|
|            1|          4|          2|   N|                N|
|            1|          5|          1|   N|                Y|
|            0|          5|          0|   N|                Y|
|            0|          3|          0|   N|                Y|
|            0|          5|          0|   N|                Y|
|            0|          5|          0|   N|           

In [10]:
df3=df2.filter(df2["total_votes"] >= 20)
df4=df3.filter(df3["helpful_votes"]/df3["total_votes"] >=0.5)
df4.show()

+-------------+-----------+-----------+----+-----------------+
|helpful_votes|star_rating|total_votes|vine|verified_purchase|
+-------------+-----------+-----------+----+-----------------+
|           30|          5|         31|   N|                N|
|           29|          5|         31|   N|                N|
|           21|          5|         24|   N|                N|
|           17|          4|         20|   N|                N|
|           22|          5|         23|   N|                N|
|           30|          4|         32|   N|                N|
|           20|          5|         20|   N|                N|
|           28|          5|         31|   N|                Y|
|           21|          5|         24|   N|                N|
|           19|          4|         25|   N|                N|
|           44|          1|         47|   N|                N|
|           27|          5|         28|   N|                Y|
|           32|          5|         32|   N|           

describe stats

In [11]:
from pyspark.sql.functions import col, avg

paid_df=df4.filter(df4['vine']=='Y')
unpaid_df=df4.filter(df4['vine']=='N')

paid_df.describe().show()
unpaid_df.describe().show()

+-------+-------------+-----------+-----------+----+-----------------+
|summary|helpful_votes|star_rating|total_votes|vine|verified_purchase|
+-------+-------------+-----------+-----------+----+-----------------+
|  count|            0|          0|          0|   0|                0|
|   mean|         null|       null|       null|null|             null|
| stddev|         null|       null|       null|null|             null|
|    min|         null|       null|       null|null|             null|
|    max|         null|       null|       null|null|             null|
+-------+-------------+-----------+-----------+----+-----------------+

+-------+------------------+------------------+------------------+-----+-----------------+
|summary|     helpful_votes|       star_rating|       total_votes| vine|verified_purchase|
+-------+------------------+------------------+------------------+-----+-----------------+
|  count|             20201|             20201|             20201|20201|            202

the summary table for vine users that rated products at 5 stars is none.

determine five star ratings from vine users



In [19]:
paid_vine_five=paid_df[paid_df["star_rating"]==5].count()
paid_count=paid_df.count()
#percent_non_vine_five=float(paid_vine_five)/float(paid_count)

print(paid_vine_five)
print(paid_count)
#print(percent_non_vine_five)

0
0


determine non vine users five star ratings

In [17]:
unpaid_vine_five=unpaid_df[unpaid_df["star_rating"]==5].count()
unpaid_count=unpaid_df.count()
percent_vine_five=float(unpaid_vine_five)/float(unpaid_count)

print(unpaid_count)
print(unpaid_vine_five)
print(percent_vine_five)

20201
12331
0.6104153259739616
